In [1]:
! pip install -U spacy -q 

     |████████████████████████████████| 6.0 MB 5.5 MB/s 
     |████████████████████████████████| 451 kB 70.7 MB/s 
     |████████████████████████████████| 10.1 MB 49.4 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 181 kB 57.0 MB/s 
     |████████████████████████████████| 628 kB 60.3 MB/s 


In [2]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.2.1                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.144+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.12                        
Pipelines                                      



In [5]:
# !wget https://raw.githubusercontent.com/amrrs/custom-ner-with-spacy/main/pvr_training_data.json

--2021-12-30 17:13:03--  https://raw.githubusercontent.com/amrrs/custom-ner-with-spacy/main/pvr_training_data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2829 (2.8K) [text/plain]
Saving to: ‘pvr_training_data.json’

pvr_training_data.j 100%[===================>]   2.76K  --.-KB/s    in 0s      

2021-12-30 17:13:03 (58.1 MB/s) - ‘pvr_training_data.json’ saved [2829/2829]



In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [7]:
import json
f = open('train.json')
TRAIN_DATA = json.load(f)

In [8]:
for text, annot in tqdm(TRAIN_DATA['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./train.spacy") # save the docbin object

100%|██████████| 1/1 [00:00<00:00, 52.63it/s]


In [9]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency


✘ The provided output file already exists. To force overwriting the
config file, set the --force or -F flag.



In [10]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    136.60    0.00    0.00    0.00    0.00
200     200       6744.75   7928.13  100.00  100.00  100.00    1.00
400     400          0.00      0.00  100.00  100.00  100.00    1.00
600     600          0.00      0.00  100.00  100.00  100.00    1.00
800     800          0.00      0.00  100.00  100.00  100.00    1.00
1000    1000          0.00      0.00  100.00  100.00  100.00    1.00
1200    1200          0.00      0.00  100.00  100.00  100.00    1.00
1400    1400          0.00      0.00  100.00  100.00  100.00    1.00
1600    1600          0.00      0

In [11]:
nlp_ner = spacy.load("model-best") 

In [16]:
doc = nlp_ner('''
Applied Cloud Computing Pvt. Ltd.
5, Anuroop CHS Samata Nagar, Pokhran Road 1 Thane Maharashtra 400606 India
Payslip for the month of July 2023 Employee Net Pay
₹9,825.00
VISHAL SINGH,
598
Paid Days : 30 | LOP Days : 0
Jr. Software Developer | Date of Joining: 13/02/2023
Bank Account No : 148801506589
EARNINGS AMOUNT YTD DEDUCTIONS AMOUNT YTD
Basic ₹2,500.00 ₹9,756.00 Withheld Salary - ₹9,825.00
House Rent Allowance ₹1,250.00 ₹4,877.00 Special Allowance
₹9,825.00 ₹9,825.00
Recovery
Petrol Allowance ₹1,000.00 ₹3,902.00
Professional Tax ₹175.00 ₹700.00
Special Allowance ₹5,250.00 ₹20,486.00
Hold Salary ₹9,825.00 ₹9,825.00
Gross Earnings ₹19,825.00 Total Deductions ₹10,000.00
Total Net Payable₹9,825.00(Indian Rupee Nine Thousand Eight Hundred Twenty-Five
Only)
**Total Net Payable = Gross Earnings - Total Deductions
-- This is a system generated payslip, hence the signature is not required. --
              Applied Cloud Computing Pvt. Ltd.
5, Anuroop CHS Samata Nagar, Pokhran Road 1 Thane Maharashtra 400606 India
Payslip for the month of July 2023 Employee Net Pay
₹9,492.00
ARVIND SHARMA,
588
Paid Days : 29 | LOP Days : 1
Digitalization Intern | Date of Joining: 13/02/2023
Bank Account No : 148801507264
EARNINGS AMOUNT YTD DEDUCTIONS AMOUNT YTD
Basic ₹2,417.00 ₹9,917.00 Professional Tax ₹175.00 ₹700.00
House Rent Allowance ₹1,208.00 ₹4,958.00
Petrol Allowance ₹967.00 ₹3,967.00
Special Allowance ₹5,075.00 ₹20,825.00
Gross Earnings ₹9,667.00 Total Deductions ₹175.00
Total Net Payable₹9,492.00(Indian Rupee Nine Thousand Four Hundred Ninety-Two
Only)
**Total Net Payable = Gross Earnings - Total Deductions
-- This is a system generated payslip, hence the signature is not required. --
''') # input sample text

In [17]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter